# Migrate a `PGVector` vector store to `PGVectorStore`

This guide shows how to migrate from the [`PGVector`](https://github.com/langchain-ai/langchain-postgres/blob/main/langchain_postgres/vectorstores.py) vector store class to the [`PGVectorStore`](https://github.com/langchain-ai/langchain-postgres/blob/main/langchain_postgres/vectorstore.py) class.

## Why migrate?

This guide explains how to migrate your vector data from a PGVector-style database (two tables) to an PGVectoStore-style database (one table per collection) for improved performance and manageability.

Migrating to the PGVectorStore interface provides the following benefits:

- **Simplified management**: A single table contains data corresponding to a single collection, making it easier to query, update, and maintain.
- **Improved metadata handling**: It stores metadata in columns instead of JSON, resulting in significant performance improvements.
- **Schema flexibility**: The interface allows users to add tables into any database schema.
- **Improved performance**: The single-table schema can lead to faster query execution, especially for large collections.
- **Clear separation**: Clearly separate table and extension creation, allowing for distinct permissions and streamlined workflows.
- **Secure Connections:** The PGVectorStore interface creates a secure connection pool that can be easily shared across your application using the `engine` object.

## Migration process

> **_NOTE:_**  The langchain-core library is installed to use the Fake embeddings service. To use a different embedding service, you'll need to install the appropriate library for your chosen provider. Choose embeddings services from [LangChain's Embedding models](https://python.langchain.com/v0.2/docs/integrations/text_embedding/).

### Library Installation
Install the integration library, `langchain-postgres`.

In [ ]:
%pip install --upgrade --quiet  langchain-postgres langchain-core SQLAlchemy

## Data Migration

### Set the postgres connection url

`PGVectorStore` can be used with the `asyncpg` and `psycopg3` drivers.

In [ ]:
# @title Set Your Values Here { display-mode: "form" }
POSTGRES_USER = "langchain"  # @param {type: "string"}
POSTGRES_PASSWORD = "langchain"  # @param {type: "string"}
POSTGRES_HOST = "localhost"  # @param {type: "string"}
POSTGRES_PORT = "6024"  # @param {type: "string"}
POSTGRES_DB = "langchain"  # @param {type: "string"}

### PGEngine Connection Pool

One of the requirements and arguments to establish PostgreSQL as a vector store is a `PGEngine` object. The `PGEngine`  configures a shared connection pool  to your Postgres database. This is an industry best practice to manage number of connections and to reduce latency through cached database connections.

To create a `PGEngine` using `PGEngine.from_connection_string()` you need to provide:

1. `url` : Connection string using the `postgresql+asyncpg` driver.


**Note:** This tutorial demonstrates the async interface. All async methods have corresponding sync methods.

In [ ]:
# See docker command above to launch a Postgres instance with pgvector enabled.
CONNECTION_STRING = (
    f"postgresql+asyncpg://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}"
    f":{POSTGRES_PORT}/{POSTGRES_DB}"
)
# To use psycopg3 driver, set your connection string to `postgresql+psycopg://`

In [ ]:
from langchain_postgres import PGEngine

engine = PGEngine.from_connection_string(url=CONNECTION_STRING)

To create a `PGEngine` using `PGEngine.from_engine()` you need to provide:

1. `engine` : An object of `AsyncEngine`

In [ ]:
from sqlalchemy.ext.asyncio import create_async_engine

# Create an SQLAlchemy Async Engine
pool = create_async_engine(
    CONNECTION_STRING,
)

engine = PGEngine.from_engine(engine=pool)

### Get all collections

This script migrates each collection to a new Vector Store table.

In [ ]:
from langchain_postgres.utils.pgvector_migrator import alist_pgvector_collection_names

all_collection_names = await alist_pgvector_collection_names(engine)
print(all_collection_names)

### Create a new table(s) to migrate existing data
The `PGVectorStore` class requires a database table. The `PGEngine` engine has a helper method `ainit_vectorstore_table()` that can be used to create a table with the proper schema for you.

You can also specify a schema name by passing `schema_name` wherever you pass `table_name`. Eg:

```python
SCHEMA_NAME="my_schema"

await engine.ainit_vectorstore_table(
    table_name=TABLE_NAME,
    vector_size=768,
    schema_name=SCHEMA_NAME,    # Default: "public"
)
```

When creating your vectorstore table, you have the flexibility to define custom metadata and ID columns. This is particularly useful for:

- **Filtering**: Metadata columns allow you to easily filter your data within the vectorstore. For example, you might store the document source, date, or author as metadata for efficient retrieval.
- **Non-UUID Identifiers**: By default, the id_column uses UUIDs. If you need to use a different type of ID (e.g., an integer or string), you can define a custom id_column.

```python
metadata_columns = [
    Column(f"col_0_{collection_name}", "VARCHAR"),
    Column(f"col_1_{collection_name}", "VARCHAR"),
]
engine.init_vectorstore_table(
    table_name="destination_table",
    vector_size=VECTOR_SIZE,
    metadata_columns=metadata_columns,
    id_column=Column("langchain_id", "VARCHAR"),
)

In [ ]:
# Vertex AI embeddings uses a vector size of 768.
# Adjust this according to your embeddings service.
VECTOR_SIZE = 768
for collection_name in all_collection_names:
    engine.init_vectorstore_table(
        table_name=collection_name,
        vector_size=VECTOR_SIZE,
    )

### Create a vector store and migrate data

> **_NOTE:_** The `FakeEmbeddings` embedding service is only used to initialize a vector store object, not to generate any embeddings. The embeddings are copied directly from the PGVector table.

If you have any customizations on the metadata or the id columns, add them to the vector store as follows:

```python
from langchain_postgres import PGVectorStore
from langchain_core.embeddings import FakeEmbeddings

destination_vector_store = PGVectorStore.create_sync(
    engine,
    embedding_service=FakeEmbeddings(size=VECTOR_SIZE),
    table_name=DESTINATION_TABLE_NAME,
    metadata_columns=[col.name for col in metadata_columns],
    id_column="langchain_id",
)
```

In [ ]:
from langchain_core.embeddings import FakeEmbeddings

from langchain_postgres import PGVectorStore
from langchain_postgres.utils.pgvector_migrator import amigrate_pgvector_collection

for collection_name in all_collection_names:
    destination_vector_store = await PGVectorStore.create(
        engine,
        embedding_service=FakeEmbeddings(size=VECTOR_SIZE),
        table_name=collection_name,
    )

    await amigrate_pgvector_collection(
        engine,
        # Set collection name here
        collection_name=collection_name,
        vector_store=destination_vector_store,
        # This deletes data from the original table upon migration. You can choose to turn it off.
        # The data will only be deleted from the original table once all of it has been successfully copied to the destination table.
        delete_pg_collection=True,
    )